# 📊 PSID Exploration & Robustness Checks

## What This Notebook Does

After finding that homeownership is associated with ~1 year more child education, we naturally wonder:

**"Is this finding robust? Does it hold across different groups? What else can we learn from the data?"**

### The Goal

This notebook explores the data more deeply and tests the robustness of our main finding through:

1. **Family Structure Analysis** - How many children did G1 families have?
2. **Subgroup Analyses** - Does the effect vary by race? By sex?
3. **Cohort Analysis** - Does the effect differ by birth decade?
4. **Missing Data Patterns** - Who's missing from our analysis?
5. **Alternative Specifications** - Different age cutoffs, model variations

**Why This Matters:**

- **Robustness** - Does our finding hold under different assumptions?
- **Generalizability** - Does the effect apply to all groups?
- **Understanding** - What factors moderate or mediate the effect?

---

## 🔑 Key Concepts

### What is a "Robustness Check"?

**Simple explanation:**
"Testing whether your main finding still holds when you change how you analyze the data."

**Example:**
- Main analysis: Age ≥ 23
- Robustness: Try age ≥ 25, age ≥ 21
- If results stay similar → Finding is "robust"

### What is a "Subgroup Analysis"?

**Simple explanation:**
"Checking if the effect differs for different groups of people."

**Example:**
- Does homeownership matter more for Black families?
- Is the effect stronger for children born in certain decades?

---

# Part 1: Setup

Same as previous notebooks - load libraries and data.

---

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Libraries loaded!")

In [ ]:
# Mount Google Drive (Colab only)
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/DATA/PSID_data

print("✅ Drive mounted!")

In [ ]:
# Load the full dataset (before sample restrictions)
df = pd.read_csv("/content/drive/MyDrive/DATA/PSID_data/parent_child_all.csv")

print(f"✅ Data loaded: {len(df):,} parent-child pairs")
print(f"   Columns: {list(df.columns)}")

---

# Part 2: Family Structure Analysis

## The Question

**"How many children did G1 families have?"**

**Why This Matters:**

Understanding family size helps us:
- Assess data completeness (Are we capturing all children?)
- Understand context (Small families? Large families?)
- Spot potential issues (Missing children?)

**What We'll Calculate:**
- Number of children per G1 parent
- Distribution of family sizes
- Average family size by homeownership status

---

## 2.1 Count Children per Parent

**How this works:**

We group by `parent_id` and count how many rows (children) each parent has.

**What `.groupby()` does:**
"Collect all rows with the same parent_id and count them."

---

In [ ]:
# Count children per parent
kids_per_parent = df.groupby('parent_id').size().reset_index(name='num_children')

print(f"✅ Analyzed {len(kids_per_parent):,} unique G1 parents")

# Summary statistics
print("\n📊 Family Size Statistics:")
print(kids_per_parent['num_children'].describe())

# Distribution
print("\n📊 Distribution of Family Sizes:")
print(kids_per_parent['num_children'].value_counts().sort_index().head(15))

## 2.2 Visualize Family Size Distribution

**What we're creating:**
A histogram showing how common different family sizes are.

**What to look for:**
- What's the most common family size?
- Are there many very large families?
- Any unusual patterns?

---

In [ ]:
# Create family size histogram
fig, ax = plt.subplots(figsize=(12, 6))

# Histogram
ax.hist(kids_per_parent['num_children'], bins=range(1, kids_per_parent['num_children'].max() + 2), 
        edgecolor='black', alpha=0.7, color='steelblue')

# Add mean line
mean_kids = kids_per_parent['num_children'].mean()
ax.axvline(mean_kids, color='red', linestyle='--', linewidth=2, 
           label=f'Mean: {mean_kids:.2f} children')

# Labels
ax.set_xlabel('Number of Children per G1 Parent', fontsize=12)
ax.set_ylabel('Number of Parents', fontsize=12)
ax.set_title('Distribution of Family Sizes (G1 → G2)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('family_size_distribution.png', dpi=300, bbox_inches='tight')
print("✅ Plot saved as 'family_size_distribution.png'")
plt.show()

## 2.3 Compare Family Size by Homeownership

**The Question:**
"Do homeowners have more or fewer children than renters?"

**Why This Matters:**
If homeowners have systematically different family sizes, that could influence our education findings.

---

In [ ]:
# Merge family size with homeownership status
if 'parent_owner' in df.columns or 'parent_V103' in df.columns:
    # Get homeownership for each parent
    if 'parent_owner' not in df.columns and 'parent_V103' in df.columns:
        df['parent_owner'] = (df['parent_V103'] == 5).astype(float)
    
    parent_homeowner = df[['parent_id', 'parent_owner']].drop_duplicates()
    kids_with_owner = kids_per_parent.merge(parent_homeowner, on='parent_id', how='left')
    
    # Compare family sizes
    print("📊 Family Size by Homeownership Status:\n")
    
    for status, label in [(0.0, "Renters"), (1.0, "Owners")]:
        group = kids_with_owner[kids_with_owner['parent_owner'] == status]['num_children']
        print(f"{label}:")
        print(f"  Mean: {group.mean():.2f} children")
        print(f"  Median: {group.median():.0f} children")
        print(f"  Std Dev: {group.std():.2f}\n")
    
    # Visual comparison
    fig, ax = plt.subplots(figsize=(10, 6))
    
    owners = kids_with_owner[kids_with_owner['parent_owner'] == 1.0]['num_children']
    renters = kids_with_owner[kids_with_owner['parent_owner'] == 0.0]['num_children']
    
    ax.hist([renters, owners], label=['Renters', 'Owners'], bins=range(1, 15), 
            alpha=0.6, color=['coral', 'steelblue'])
    ax.set_xlabel('Number of Children', fontsize=12)
    ax.set_ylabel('Frequency', fontsize=12)
    ax.set_title('Family Size by Homeownership Status', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  Homeownership variable not found - cannot compare family sizes")

---

# Part 3: Subgroup Analyses

## The Goal

Test whether the homeownership effect differs across:
- **Race groups** (White, Black, Other/Hispanic)
- **Sex** (Male vs. Female children)
- **Birth cohorts** (Different decades)

**Why This Matters:**

If the effect is much stronger (or only present) in certain groups, that tells us:
- Who benefits most from homeownership
- Whether the effect is universal
- Potential mechanisms

---

## 3.1 Prepare Analysis Sample

Use the same sample restrictions as Notebook 02.

---

In [ ]:
# Create analysis sample (same as Notebook 02)
# Note: Adjust these filters based on your actual variable names

# Create binary homeownership if needed
if 'parent_owner' not in df.columns and 'parent_V103' in df.columns:
    df['parent_owner'] = (df['parent_V103'] == 5).astype(float)

# Apply filters
analysis_sample = df[
    (df.get('observable', pd.Series([True]*len(df))) == True) &
    (df['parent_owner'].notna()) &
    (df.get('child_education_years', pd.Series([np.nan]*len(df))).notna()) &
    (df.get('child_sex', pd.Series([np.nan]*len(df))).notna()) &
    (df.get('child_race', pd.Series([np.nan]*len(df))).notna())
].copy()

print(f"✅ Analysis sample: {len(analysis_sample):,} children")

## 3.2 Subgroup Analysis by Race

**The Question:**
"Does homeownership matter more for some racial groups than others?"

**How we test this:**
Run separate regressions for each race group.

**What to look for:**
- Are coefficients similar across groups?
- Is the effect significant in all groups?
- Which group benefits most?

---

In [ ]:
# Subgroup analysis by race
if 'child_race' in analysis_sample.columns and 'child_education_years' in analysis_sample.columns:
    print("=" * 80)
    print("📊 SUBGROUP ANALYSIS: BY RACE")
    print("=" * 80)
    
    race_labels = {1.0: "White", 2.0: "Black", 3.0: "Other/Hispanic"}
    results = []
    
    for race_code, race_name in race_labels.items():
        subgroup = analysis_sample[analysis_sample['child_race'] == race_code]
        
        if len(subgroup) < 100:  # Skip if too few observations
            print(f"\n⚠️  {race_name}: Too few observations ({len(subgroup)})")
            continue
        
        # Run regression
        model = ols('child_education_years ~ parent_owner', data=subgroup).fit()
        
        coef = model.params['parent_owner']
        pval = model.pvalues['parent_owner']
        n = len(subgroup)
        
        results.append({
            'Race': race_name,
            'N': n,
            'Coefficient': coef,
            'P-value': pval
        })
        
        sig = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else "n.s."
        
        print(f"\n{race_name}:")
        print(f"  N: {n:,}")
        print(f"  Homeownership Effect: {coef:.3f} years {sig}")
        print(f"  P-value: {pval:.4f}")
    
    # Summary comparison
    if results:
        results_df = pd.DataFrame(results)
        print("\n" + "=" * 80)
        print("📊 SUMMARY COMPARISON:")
        print(results_df.to_string(index=False))
        print("\n💡 Interpretation: Compare the coefficients across groups.")
        print("   Larger differences suggest the effect varies by race.")
    
    print("\n" + "=" * 80)
else:
    print("⚠️  Required variables not found for race subgroup analysis")

## 3.3 Subgroup Analysis by Sex

**The Question:**
"Does homeownership matter more for boys or girls?"

---

In [ ]:
# Subgroup analysis by sex
if 'child_sex' in analysis_sample.columns and 'child_education_years' in analysis_sample.columns:
    print("=" * 80)
    print("📊 SUBGROUP ANALYSIS: BY SEX")
    print("=" * 80)
    
    sex_labels = {1.0: "Male", 2.0: "Female"}
    
    for sex_code, sex_name in sex_labels.items():
        subgroup = analysis_sample[analysis_sample['child_sex'] == sex_code]
        
        # Run regression
        model = ols('child_education_years ~ parent_owner', data=subgroup).fit()
        
        coef = model.params['parent_owner']
        pval = model.pvalues['parent_owner']
        sig = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else "n.s."
        
        print(f"\n{sex_name}:")
        print(f"  N: {len(subgroup):,}")
        print(f"  Homeownership Effect: {coef:.3f} years {sig}")
        print(f"  P-value: {pval:.4f}")
    
    print("\n" + "=" * 80)
else:
    print("⚠️  Required variables not found for sex subgroup analysis")

## 3.4 Cohort Analysis by Birth Decade

**The Question:**
"Did the homeownership effect change over time?"

**Why This Matters:**

- Educational opportunities changed dramatically across decades
- Economic conditions varied
- Homeownership may have mattered more in some eras

---

In [ ]:
# Cohort analysis by birth decade
if 'birth_decade' in analysis_sample.columns and 'child_education_years' in analysis_sample.columns:
    print("=" * 80)
    print("📊 COHORT ANALYSIS: BY BIRTH DECADE")
    print("=" * 80)
    
    decades = sorted(analysis_sample['birth_decade'].dropna().unique())
    results = []
    
    for decade in decades:
        subgroup = analysis_sample[analysis_sample['birth_decade'] == decade]
        
        if len(subgroup) < 50:  # Skip if too few
            continue
        
        # Run regression
        model = ols('child_education_years ~ parent_owner', data=subgroup).fit()
        
        coef = model.params['parent_owner']
        pval = model.pvalues['parent_owner']
        n = len(subgroup)
        
        results.append({
            'Decade': int(decade),
            'N': n,
            'Coefficient': coef,
            'P-value': pval
        })
        
        sig = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else "n.s."
        
        print(f"\n{int(decade)}s:")
        print(f"  N: {n:,}")
        print(f"  Homeownership Effect: {coef:.3f} years {sig}")
    
    # Plot trends over time
    if results:
        results_df = pd.DataFrame(results)
        
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(results_df['Decade'], results_df['Coefficient'], 
                marker='o', linewidth=2, markersize=10, color='steelblue')
        ax.axhline(0, color='red', linestyle='--', alpha=0.5)
        ax.set_xlabel('Birth Decade', fontsize=12)
        ax.set_ylabel('Homeownership Effect (Years)', fontsize=12)
        ax.set_title('Homeownership Effect by Birth Cohort', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('cohort_analysis.png', dpi=300, bbox_inches='tight')
        print("\n✅ Plot saved as 'cohort_analysis.png'")
        plt.show()
    
    print("\n" + "=" * 80)
else:
    print("⚠️  birth_decade variable not found")

---

# Part 4: Missing Data Analysis

## The Goal

Understand who's missing from our analysis and whether missingness is systematic.

**Why This Matters:**

If data is "missing at random," our results are fine. But if missingness is systematic (e.g., all homeowners are missing education data), we could have biased results.

---

## 4.1 Missing Data Patterns

**What we're checking:**
- Which variables have missing data?
- How much is missing?
- Is missingness related to homeownership?

---

In [ ]:
# Analyze missing data patterns
print("=" * 80)
print("📊 MISSING DATA ANALYSIS")
print("=" * 80)

# Overall missingness
key_vars = ['parent_owner', 'child_education_years', 'child_race', 'child_sex', 
            'child_age', 'parent_education']

print("\n📋 Missing Data by Variable:\n")
missing_summary = []

for var in key_vars:
    if var in df.columns:
        n_missing = df[var].isna().sum()
        pct_missing = (n_missing / len(df)) * 100
        missing_summary.append({
            'Variable': var,
            'N Missing': n_missing,
            'Percent Missing': f"{pct_missing:.1f}%"
        })
        print(f"{var:25s}: {n_missing:7,} missing ({pct_missing:5.1f}%)")

print("\n" + "=" * 80)

## 4.2 Check if Missingness Relates to Homeownership

**Critical question:**
"Is education data more likely to be missing for homeowners vs. renters?"

If yes → potential bias
If no → probably okay

---

In [ ]:
# Check if missingness relates to homeownership
if 'parent_owner' in df.columns and 'child_education_years' in df.columns:
    print("=" * 80)
    print("📊 DOES MISSINGNESS RELATE TO HOMEOWNERSHIP?")
    print("=" * 80)
    
    # Create missingness indicator
    df['education_missing'] = df['child_education_years'].isna()
    
    # Compare missingness rates
    missing_by_owner = df.groupby('parent_owner')['education_missing'].mean() * 100
    
    print("\n📋 Education Missing Rate by Homeownership:\n")
    print(f"  Renters (0): {missing_by_owner.get(0.0, 0):.1f}% missing")
    print(f"  Owners (1): {missing_by_owner.get(1.0, 0):.1f}% missing")
    
    difference = abs(missing_by_owner.get(1.0, 0) - missing_by_owner.get(0.0, 0))
    
    if difference < 5:
        print("\n✅ GOOD: Missing rates are similar between groups")
        print("   Missingness appears to be independent of homeownership")
    else:
        print("\n⚠️  WARNING: Missing rates differ substantially")
        print("   This could introduce bias into our estimates")
    
    print("\n" + "=" * 80)

---

# Part 5: Robustness Checks

## The Goal

Test whether our main finding holds under different assumptions.

**Tests we'll run:**
1. Different age cutoffs (21, 23, 25)
2. Include/exclude certain variables
3. Different sample restrictions

---

## 5.1 Alternative Age Cutoffs

**The Question:**
"Does our finding change if we use different age cutoffs for 'observable' education?"

**Main analysis:** Age ≥ 23
**Robustness:** Try age ≥ 21, 25, 30

---

In [ ]:
# Test different age cutoffs
if 'child_age' in df.columns and 'child_education_years' in df.columns:
    print("=" * 80)
    print("📊 ROBUSTNESS: ALTERNATIVE AGE CUTOFFS")
    print("=" * 80)
    
    age_cutoffs = [21, 23, 25, 30]
    results = []
    
    for age_cutoff in age_cutoffs:
        # Create sample with this age cutoff
        robust_sample = df[
            (df['child_age'] >= age_cutoff) &
            (df['parent_owner'].notna()) &
            (df['child_education_years'].notna())
        ]
        
        if len(robust_sample) < 100:
            continue
        
        # Run regression
        model = ols('child_education_years ~ parent_owner', data=robust_sample).fit()
        
        coef = model.params['parent_owner']
        pval = model.pvalues['parent_owner']
        n = len(robust_sample)
        
        results.append({
            'Age Cutoff': age_cutoff,
            'N': n,
            'Coefficient': coef,
            'P-value': pval
        })
        
        sig = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else "n.s."
        marker = "⭐" if age_cutoff == 23 else "  "
        
        print(f"\n{marker} Age ≥ {age_cutoff}:")
        print(f"   N: {n:,}")
        print(f"   Coefficient: {coef:.3f} years {sig}")
    
    # Summary
    if results:
        results_df = pd.DataFrame(results)
        print("\n" + "=" * 80)
        print("📊 SUMMARY:")
        print(results_df.to_string(index=False))
        print("\n💡 If coefficients are similar across cutoffs → Finding is ROBUST")
        print("   ⭐ = Main analysis (age ≥ 23)")
    
    print("\n" + "=" * 80)
else:
    print("⚠️  Required variables not found for age robustness check")

---

# 🎯 Summary: What We Learned

## Family Structure
- Average family size: ~X children per G1 parent
- Most common family size: X children
- Homeowners vs. renters: Similar/different family sizes

## Subgroup Findings
- **By Race:** Effect is [similar/different] across racial groups
- **By Sex:** Effect is [similar/different] for boys vs. girls
- **By Cohort:** Effect [strengthened/weakened/stayed stable] over time

## Robustness
- **Missing Data:** [Random/systematic] missingness patterns
- **Age Cutoffs:** Finding [is/is not] robust to different age restrictions

## Overall Assessment

✅ **If all checks pass:** 
"The homeownership effect appears robust. It holds across different groups, time periods, and analytical choices. This strengthens our confidence in the finding."

⚠️ **If some checks fail:**
"The homeownership effect may be [specific to certain groups/sensitive to modeling choices/affected by missing data]. Interpret with appropriate caveats."

---

## Files Generated

- `family_size_distribution.png` - Family size histogram
- `cohort_analysis.png` - Trends over birth decades
- Various diagnostic outputs

---

# End of Notebook 03

**Status:** ✅ Exploration & Robustness Complete  
**Next:** (Optional) Proceed to `04_Three_Generations.ipynb`

---